This was adapted from the pyCaret Multiclass Classification Tutorial.
https://github.com/pycaret/pycaret/blob/master/tutorials/Tutorial%20-%20Multiclass%20Classification.ipynb

I left in most of the help text/explanations.


In [1]:
# check installed version
import pycaret
pycaret.__version__

'3.1.0'

PyCaret’s Classification Module is a supervised machine learning module that is used for classifying elements into groups. The goal is to predict the categorical class labels which are discrete and unordered. 

Some common use cases include predicting customer default (Yes or No), predicting customer churn (customer will leave or stay), the disease found (positive or negative). 

This module can be used for binary or multiclass problems. It provides several pre-processing features that prepare the data for modeling through the setup function. It has over 18 ready-to-use algorithms and several plots to analyze the performance of trained models.

A typical workflow in PyCaret consist of following 5 steps in this order:

## **Setup** ➡️ **Compare Models** ➡️ **Analyze Model** ➡️ **Prediction** ➡️ **Save Model**

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pycaret.classification import *
from imblearn.over_sampling import *
from imblearn.under_sampling import *
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np

In [3]:
data = pd.read_csv("..\diabetes_cleaned.csv")

In [4]:
data.drop(['Unnamed: 0','heart_attack','angina_or_chd','chd_mi'], axis=1, inplace=True)

#data.drop(['income','age','sleep_time'], axis=1, inplace=True)

In [5]:
TARGET = 'diabetes'
ANSWER = 42

In [6]:
data.diabetes.value_counts()

0.0    254681
2.0     41479
1.0      6793
Name: diabetes, dtype: int64

In [7]:
#split data into 20% split with equal parts diabetes labels.
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42, stratify=data['diabetes'])

In [8]:
# The unbalanced ratio
print("Train")
print(train_df.diabetes.value_counts())
print("Test")
test_df.diabetes.value_counts()

Train
0.0    203745
2.0     33183
1.0      5434
Name: diabetes, dtype: int64
Test


0.0    50936
2.0     8296
1.0     1359
Name: diabetes, dtype: int64

In [9]:
test_labels = test_df[TARGET]
test_features = test_df.drop(columns=TARGET)

In [10]:
# #binary
# binary_data = train_df.copy()
# binary_data.loc[binary_data.diabetes.isin([2]), TARGET] = 1
# binary_data.diabetes.value_counts()
# data = binary_data

In [11]:
# over_sampled_data = OverSample(train_df, TARGET, "RandomOverSampler")

# print(over_sampled_data.diabetes.value_counts())

In [12]:
# over_sampled_binary_data = OverSample(binary_data, TARGET, "RandomOverSampler")

# print(over_sampled_binary_data.diabetes.value_counts())

In [13]:
def Sampler(df, target, sampler_name, stragegy):
    sampler = globals()[sampler_name](sampling_strategy=strategy)

    under_train_labels = df[target]
    under_train_features = df.drop(columns=target)
    
    X, y = sampler.fit_resample(under_train_features, under_train_labels)
    
    sampled_data = X
    sampled_data[target] = y
    
    print(sampled_data[target].value_counts())
    return sampled_data    

In [14]:
# def UnderSample(df, target, sampler_name):
#     sampler = globals()[sampler_name]
#     under_sampler = sampler()

#     under_train_labels = df[target]
#     under_train_features = df.drop(columns=target)
    
#     X_under, y_under = under_sampler.fit_resample(under_train_features, under_train_labels)
    
#     under_sampled_data = X_under
#     under_sampled_data[TARGET] = y_under
#     return under_sampled_data    

# def OverSample(df, target, sampler_name):
#     sampler = globals()[sampler_name]
#     over_sampler = sampler(random_state=ANSWER)
    
#     train_labels = df[target].copy()
#     train_features = df.drop(columns=target)

#     over_sampled_features, over_sampled_labels = over_sampler.fit_resample(train_features, train_labels)

#     over_sampled_data = over_sampled_features
#     over_sampled_data[target] = over_sampled_labels
    
#     return over_sampled_data

In [15]:
# under_sampled_data = UnderSample(train_df, TARGET, "RandomUnderSampler")
# print(under_sampled_data.diabetes.value_counts())

In [16]:
# under_sampled_binary_data = UnderSample(binary_data, TARGET, "RandomUnderSampler")

# print(under_sampled_binary_data.diabetes.value_counts())

In [17]:
# s = setup(under_sampled_data, target = TARGET, session_id = 123)
# best = compare_models()

In [18]:
#datasets = [under_sampled_data, over_sampled_data, under_sampled_binary_data, over_sampled_binary_data]
# datasets = [over_sampled_data]
# for idx, data in enumerate(datasets):

if False:
    over_samplers = ['RandomOverSampler','SMOTE','SMOTEN','ADASYN','BorderlineSMOTE','KMeansSMOTE','SVMSMOTE']
    for sampler in over_samplers:
        over_sampled_data = OverSample(train_df, TARGET, sampler)
        s = setup(over_sampled_data, target = TARGET, session_id = 123)
        best = compare_models()
        test_acc = best.score(test_features,  test_labels)

        print(f'\nSampler: {sampler} Test accuracy:', test_acc)


In [19]:
#  test_acc = best.score(test_features,  test_labels)
# print(f'\nSampler: {sampler}, Strategy: {strategy}, Test accuracy:', test_acc)

In [20]:
# sampled_data.diabetes.value_counts()

In [21]:
import matplotlib.pyplot as plt

def print_test_heat_map(model, sampler, strategy):
    predictions = predict_model(model, data=test_features)
    cm = confusion_matrix(test_labels, predictions["prediction_label"])
    cm_total = cm.sum(axis=1)
    result = [[0.0,0.0,0.0],[0.0,0.0,0.0],[0.0,0.0,0.0]]    
    for row in range(0,3):
        for column in range(0,3):
            result[row][column] = cm[row][column]/(cm_total[row])
            
    total_percent = result[0][0] + result[1][1] + result[2][2]
    diabetes_percent = result[1][1] + result[2][2]
    # print(f"Total Percent Accurate {total_percent*100}")
    # print(f"Diabetes Percent Accurate {diabetes_percent*100}")
    
    print(f"{type(model).__name__},{sampler},{strategy},{total_percent*100},{diabetes_percent*100}")
    # sns.heatmap(result, annot=True, 
    #             fmt='.2%', cmap='Blues')
    # plt.show()
    # plt.figure()


In [22]:
print(f"{100:.2f}")

100.00


In [23]:
samplers = ['SMOTE','ADASYN']
# samplers = ['RandomOverSampler', 'EditedNearestNeighbours','RepeatedEditedNearestNeighbours','AllKNN','NeighbourhoodCleaningRule','OneSidedSelection','TomekLinks', 'SMOTE','ADASYN']

# under_samplers = ['RepeatedEditedNearestNeighbours',]
# over_samplers = ['RandomOverSampler', 'SMOTE','ADASYN']

# working_sampler = ['SMOTE']

#samplers = ['TomekLinks','SMOTE','RandomOverSampler','NearMiss']

for sampler in samplers:
    strategies = ['auto']
    for strategy in strategies:
        sampled_data = Sampler(train_df, TARGET, sampler, strategy)
        sampled_data.diabetes.value_counts()
        s = setup(sampled_data, target = TARGET, session_id = 123,verbose=False)
        best = compare_models(n_select=14,verbose=False)
        for model in best:    

            #print(f'\nSampler: {sampler}, Strategy: {strategy}')
            print_test_heat_map(model, sampler, strategy)
            
        # test_acc = best.score(test_features,  test_labels)


        # #evaluate_model(best)
        # plot_model(best, plot = 'auc')
        # plot_model(best, plot = 'confusion_matrix', plot_kwargs = {'percent' : True})
        # print_test_heat_map()
        # plot_model(best, plot = 'feature')


0.0    203745
2.0    203745
1.0    203745
Name: diabetes, dtype: int64


ExtraTreesClassifier,SMOTE,auto,119.35053829999592,27.08966190608983


RandomForestClassifier,SMOTE,auto,119.37430394398785,26.398805278996484


DecisionTreeClassifier,SMOTE,auto,119.58443974774471,34.00056979329205


KNeighborsClassifier,SMOTE,auto,133.85153991715774,69.57087398343697


...

RidgeClassifier,ADASYN,auto,153.31122394979778,86.43318091540166


GaussianNB,ADASYN,auto,150.96207517200153,92.6791318706037


SGDClassifier,ADASYN,auto,148.08539275562035,77.71473153369479


QuadraticDiscriminantAnalysis,ADASYN,auto,100.0,0.0


DummyClassifier,ADASYN,auto,100.0,0.0


TomekLinks,LGBMClassifier,all,109.852,11.307
TomekLinks,GradientBoostingClassifier,all,111.113,12.705
TomekLinks,AdaBoostClassifier,all,112.107,14.103
TomekLinks,LogisticRegression,all,110.422,12.283
TomekLinks,RidgeClassifier,all,100.808,0.904
TomekLinks,QuadraticDiscriminantAnalysis,all,100.000,0.000
TomekLinks,DummyClassifier,all,100.000,0.000
TomekLinks,SGDClassifier,all,100.000,0.000
TomekLinks,LinearDiscriminantAnalysis,all,113.627,16.225
TomekLinks,RandomForestClassifier,all,111.796,15.397
TomekLinks,KNeighborsClassifier,all,112.120,15.986
TomekLinks,ExtraTreesClassifier,all,111.944,16.183
TomekLinks,DecisionTreeClassifier,all,117.975,29.949
TomekLinks,GaussianNB,all,135.438,51.189
SMOTE,ExtraTreesClassifier,all,119.503,27.078
SMOTE,RandomForestClassifier,all,119.470,26.481
SMOTE,DecisionTreeClassifier,all,119.515,34.080
SMOTE,KNeighborsClassifier,all,133.997,69.761
SMOTE,LGBMClassifier,all,126.280,32.739
SMOTE,GradientBoostingClassifier,all,143.294,60.205
SMOTE,AdaBoostClassifier,all,146.407,69.632
SMOTE,LogisticRegression,all,155.449,91.614
SMOTE,LinearDiscriminantAnalysis,all,154.212,92.069
SMOTE,RidgeClassifier,all,153.040,85.675
SMOTE,GaussianNB,all,150.418,91.936
SMOTE,SGDClassifier,all,149.297,73.918
SMOTE,QuadraticDiscriminantAnalysis,all,100.000,0.000
SMOTE,DummyClassifier,all,100.000,0.000
RandomOverSampler,ExtraTreesClassifier,all,120.842,32.939
RandomOverSampler,RandomForestClassifier,all,130.257,46.356
RandomOverSampler,DecisionTreeClassifier,all,121.653,42.809
RandomOverSampler,KNeighborsClassifier,all,134.672,70.946
RandomOverSampler,LGBMClassifier,all,156.799,94.474
RandomOverSampler,GradientBoostingClassifier,all,157.825,95.541
RandomOverSampler,AdaBoostClassifier,all,156.259,92.057
RandomOverSampler,LogisticRegression,all,155.351,90.884
RandomOverSampler,LinearDiscriminantAnalysis,all,154.363,91.478
RandomOverSampler,RidgeClassifier,all,152.595,84.325
RandomOverSampler,SGDClassifier,all,151.578,84.476
RandomOverSampler,GaussianNB,all,147.883,78.507
RandomOverSampler,QuadraticDiscriminantAnalysis,all,100.000,0.000
RandomOverSampler,DummyClassifier,all,100.000,0.000
NearMiss,LGBMClassifier,all,107.868,96.693
NearMiss,ExtraTreesClassifier,all,103.376,95.590
NearMiss,RandomForestClassifier,all,104.406,95.723
NearMiss,GradientBoostingClassifier,all,111.298,94.143
NearMiss,AdaBoostClassifier,all,110.956,94.993
NearMiss,LogisticRegression,all,113.049,92.922
NearMiss,DecisionTreeClassifier,all,104.450,92.025
NearMiss,LinearDiscriminantAnalysis,all,118.792,89.225
NearMiss,RidgeClassifier,all,119.180,88.446
NearMiss,KNeighborsClassifier,all,113.472,85.859
NearMiss,SGDClassifier,all,108.533,88.500
NearMiss,GaussianNB,all,113.814,80.816
NearMiss,QuadraticDiscriminantAnalysis,all,100.000,0.000
NearMiss,DummyClassifier,all,100.000,0.000
RandomOverSampler,ExtraTreesClassifier,auto,120.842,32.939
RandomOverSampler,RandomForestClassifier,auto,130.257,46.356
RandomOverSampler,DecisionTreeClassifier,auto,121.653,42.809
RandomOverSampler,KNeighborsClassifier,auto,134.672,70.946
RandomOverSampler,LGBMClassifier,auto,156.799,94.474
RandomOverSampler,GradientBoostingClassifier,auto,157.825,95.541
RandomOverSampler,AdaBoostClassifier,auto,156.259,92.057
RandomOverSampler,LogisticRegression,auto,155.351,90.884
RandomOverSampler,LinearDiscriminantAnalysis,auto,154.363,91.478
RandomOverSampler,RidgeClassifier,auto,152.595,84.325
RandomOverSampler,SGDClassifier,auto,151.578,84.476
RandomOverSampler,GaussianNB,auto,147.883,78.507
RandomOverSampler,QuadraticDiscriminantAnalysis,auto,100.000,0.000
RandomOverSampler,DummyClassifier,auto,100.000,0.000
RandomForestClassifier,AllKNN,auto,102.73105732810244,3.830476206695178
ExtraTreesClassifier,AllKNN,auto,102.42930781476834,3.7702061970519765
KNeighborsClassifier,AllKNN,auto,102.79987575252191,3.642109143443865
GradientBoostingClassifier,AllKNN,auto,102.77590582407268,3.0978784956605594
LGBMClassifier,AllKNN,auto,103.79888540311583,4.256322186530313
AdaBoostClassifier,AllKNN,auto,102.95808842272088,3.3389585342333654
RidgeClassifier,AllKNN,auto,100.0,0.0
QuadraticDiscriminantAnalysis,AllKNN,auto,100.0,0.0
DummyClassifier,AllKNN,auto,100.0,0.0
LogisticRegression,AllKNN,auto,101.75010454036509,1.9660618200886548
SGDClassifier,AllKNN,auto,100.09474328137536,0.10848601735776277
LinearDiscriminantAnalysis,AllKNN,auto,111.2661147257761,13.319672131147541
DecisionTreeClassifier,AllKNN,auto,110.31092974161871,17.19604046880577
GaussianNB,AllKNN,auto,132.41211590536,46.65548012712848
RandomForestClassifier,AllKNN,auto,102.73105732810244,3.830476206695178
ExtraTreesClassifier,AllKNN,auto,102.42930781476834,3.7702061970519765
KNeighborsClassifier,AllKNN,auto,102.79987575252191,3.642109143443865
GradientBoostingClassifier,AllKNN,auto,102.77590582407268,3.0978784956605594
LGBMClassifier,AllKNN,auto,103.79888540311583,4.256322186530313
AdaBoostClassifier,AllKNN,auto,102.95808842272088,3.3389585342333654
RidgeClassifier,AllKNN,auto,100.0,0.0
QuadraticDiscriminantAnalysis,AllKNN,auto,100.0,0.0
DummyClassifier,AllKNN,auto,100.0,0.0
LogisticRegression,AllKNN,auto,101.75010454036509,1.9660618200886548
SGDClassifier,AllKNN,auto,100.09474328137536,0.10848601735776277
LinearDiscriminantAnalysis,AllKNN,auto,111.2661147257761,13.319672131147541
DecisionTreeClassifier,AllKNN,auto,110.31092974161871,17.19604046880577
GaussianNB,AllKNN,auto,132.41211590536,46.65548012712848
ExtraTreesClassifier,SMOTE,auto,119.35053829999592,27.08966190608983
RandomForestClassifier,SMOTE,auto,119.37430394398785,26.398805278996484
DecisionTreeClassifier,SMOTE,auto,119.58443974774471,34.00056979329205
KNeighborsClassifier,SMOTE,auto,133.85153991715774,69.57087398343697
LGBMClassifier,SMOTE,auto,125.91517647876687,32.34088717454195
GradientBoostingClassifier,SMOTE,auto,143.1386250630391,59.56512105801318
AdaBoostClassifier,SMOTE,auto,147.91058443553914,73.11868872327275
LogisticRegression,SMOTE,auto,154.92010532501956,90.9457060789068
LinearDiscriminantAnalysis,SMOTE,auto,153.7499042307075,91.53653843834063
RidgeClassifier,SMOTE,auto,153.31489420770313,85.86358275804078
GaussianNB,SMOTE,auto,150.16552075782676,91.4899278569315
SGDClassifier,SMOTE,auto,152.92505514173868,94.59303064040367
QuadraticDiscriminantAnalysis,SMOTE,auto,100.0,0.0
DummyClassifier,SMOTE,auto,100.0,0.0
ExtraTreesClassifier,ADASYN,auto,118.23928912629312,25.78405118063583
RandomForestClassifier,ADASYN,auto,118.3693811748641,25.311426093978284
DecisionTreeClassifier,ADASYN,auto,119.26898880194202,33.63211115155721
KNeighborsClassifier,ADASYN,auto,132.68336694531905,69.70040793793724
LGBMClassifier,ADASYN,auto,123.20348419912885,28.700578592092572
GradientBoostingClassifier,ADASYN,auto,143.2988719950276,60.50281419700656
AdaBoostClassifier,ADASYN,auto,145.78613072514108,68.32029124029737
LogisticRegression,ADASYN,auto,153.81145063441457,90.14528132390727
LinearDiscriminantAnalysis,ADASYN,auto,154.12691534526903,92.01563844877147
RidgeClassifier,ADASYN,auto,153.31122394979778,86.43318091540166
GaussianNB,ADASYN,auto,150.96207517200153,92.6791318706037
SGDClassifier,ADASYN,auto,148.08539275562035,77.71473153369479
QuadraticDiscriminantAnalysis,ADASYN,auto,100.0,0.0
DummyClassifier,ADASYN,auto,100.0,0.0

In [ ]:
cm_total

In [ ]:

predictions = predict_model(best, data=test_features)
cm = confusion_matrix(test_labels, predictions["prediction_label"])
cm_total = cm.sum(axis=1)
result = [[0.0,0.0,0.0],[0.0,0.0,0.0],[0.0,0.0,0.0]]    
for row in range(0,3):
    for column in range(0,3):
        result[row][column] = cm[row][column]/(cm_total[row])
        
sns.heatmap(result, annot=True, 
            fmt='.2%', cmap='Blues')

In [ ]:
predictions = predict_model(best, data=test_features)
predictions.head()

In [ ]:
plot_model(best, plot = 'confusion_matrix')

In [ ]:
plot_model(best, plot = 'boundary')

In [ ]:
plot_model(best, plot = 'class_report')

In [ ]:
plot_model(best, plot = 'feature')

In [ ]:
plot_model(best, plot = 'learning')

## Setup
This function initializes the training environment and creates the transformation pipeline. Setup function must be called before executing any other function in PyCaret. It only has two required parameters i.e. `data` and `target`. All the other parameters are optional.

In [ ]:
# import pycaret classification and init setup



Once the setup has been successfully executed it shows the information grid containing experiment level information. 

- **Session id:**  A pseudo-random number distributed as a seed in all functions for later reproducibility. If no `session_id` is passed, a random number is automatically generated that is distributed to all functions.<br/>
<br/>
- **Target type:**  Binary, Multiclass, or Regression. The Target type is automatically detected. <br/>
<br/>
- **Label Encoding:**  When the Target variable is of type string (i.e. 'Yes' or 'No') instead of 1 or 0, it automatically encodes the label into 1 and 0 and displays the mapping (0 : No, 1 : Yes) for reference. In this tutorial, no label encoding is required since the target variable is of numeric type. <br/>
<br/>
- **Original data shape:**  Shape of the original data prior to any transformations. <br/>
<br/>
- **Transformed train set shape :**  Shape of transformed train set <br/>
<br/>
- **Transformed test set shape :**  Shape of transformed test set <br/>
<br/>
- **Numeric features :**  The number of features considered as numerical. <br/>
<br/>
- **Categorical features :**  The number of features considered as categorical. <br/>

PyCaret has two set of API's that you can work with. (1) Functional (as seen above) and (2) Object Oriented API.

With Object Oriented API instead of executing functions directly you will import a class and execute methods of class.

You can use any of the two method i.e. Functional or OOP and even switch back and forth between two set of API's. The choice of method will not impact the results and has been tested for consistency.

## Compare Models

This function trains and evaluates the performance of all the estimators available in the model library using cross-validation. The output of this function is a scoring grid with average cross-validated scores. Metrics evaluated during CV can be accessed using the `get_metrics` function. Custom metrics can be added or removed using `add_metric` and `remove_metric` function.

In [ ]:
# compare baseline models
best = compare_models()

## Analyze Model

You can use the `plot_model` function to analyzes the performance of a trained model on the test set. It may require re-training the model in certain cases.

In [ ]:
sampled_data.diabetes.value_counts()

In [ ]:
# plot confusion matrix
plot_model(best, plot = 'confusion_matrix')

In [ ]:
# plot AUC
plot_model(best, plot = 'auc')

In [ ]:
# plot feature importance
plot_model(best, plot = 'feature')

In [ ]:
# check docstring to see available plots 
help(plot_model)

An alternate to `plot_model` function is `evaluate_model`. It can only be used in Notebook since it uses ipywidget.

In [ ]:
evaluate_model(best)

## Prediction
The `predict_model` function returns `prediction_label` and `prediction_score` (probability of the predicted class) as new columns in dataframe. When data is `None` (default), it uses the test set (created during the setup function) for scoring.

In [ ]:
# predict on test set
holdout_pred = predict_model(best)

In [ ]:
# show predictions df
holdout_pred.head()

The same function works for predicting the labels on unseen dataset. Let's create a copy of original data and drop the `Class variable`. We can then use the new data frame without labels for scoring.

In [ ]:
# copy data and drop Class variable

new_data = data.copy()
new_data.drop('diabetes', axis=1, inplace=True)
new_data.head()

In [ ]:
# predict model on new_data
predictions = predict_model(best, data = new_data)
predictions.head()

## Save Model

Finally, you can save the entire pipeline on disk for later use, using pycaret's `save_model` function.

In [ ]:
# save pipeline
save_model(best, 'my_first_pipeline')

In [ ]:
# load pipeline
loaded_best_pipeline = load_model('my_first_pipeline')
loaded_best_pipeline

In [ ]:
# check all the available models
models()

In [ ]:
# train logistic regression with default fold=10
lr = create_model('lr')

The function above has return trained model object as an output. The scoring grid is only displayed and not returned. If you need access to the scoring grid you can use `pull` function to access the dataframe.

In [ ]:
lr_results = pull()
print(type(lr_results))
lr_results

In [ ]:
# train logistic regression with fold=3
lr = create_model('lr', fold=3)

In [ ]:
# train logistic regression with specific model parameters
create_model('lr', C = 0.5, l1_ratio = 0.15)

In [ ]:
# train lr and return train score as well alongwith CV
create_model('lr', return_train_score=True)

Some other parameters that you might find very useful in `create_model` are:

- cross_validation
- engine
- fit_kwargs
- groups

You can check the docstring of the function for more info.

In [ ]:
# help(create_model)

## ✅ Tune Model

This function tunes the hyperparameters of the model. The output of this function is a scoring grid with cross-validated scores by fold. The best model is selected based on the metric defined in optimize parameter. Metrics evaluated during cross-validation can be accessed using the `get_metrics` function. Custom metrics can be added or removed using `add_metric` and `remove_metric` function.

In [ ]:
# train a dt model with default params
dt = create_model('dt')

In [ ]:
# tune hyperparameters of dt
tuned_dt = tune_model(dt)

Metric to optimize can be defined in `optimize` parameter (default = 'Accuracy'). Also, a custom tuned grid can be passed with `custom_grid` parameter. 

In [ ]:
dt

In [ ]:
# define tuning grid
dt_grid = {'max_depth' : [None, 2, 4, 6, 8, 10, 12]}

# tune model with custom grid and metric = F1
tuned_dt = tune_model(dt, custom_grid = dt_grid, optimize = 'F1')

In [ ]:
# to access the tuner object you can set return_tuner = True
tuned_dt, tuner = tune_model(dt, return_tuner=True)

In [ ]:
# model object
tuned_dt

In [ ]:
# tuner object
tuner

The default search algorithm is `RandomizedSearchCV` from `sklearn`. This can be changed by using `search_library` and `search_algorithm` parameter.

In [ ]:
# tune dt using optuna
tuned_dt = tune_model(dt, search_library = 'optuna')

For more details on all available `search_library` and `search_algorithm` please check the docstring. Some other parameters that you might find very useful in `tune_model` are:

- choose_better
- n_iter
- early_stopping
- groups

You can check the docstring of the function for more info.

In [ ]:
# help(tune_model)

## ✅ Ensemble Model

This function ensembles a given estimator. The output of this function is a scoring grid with CV scores by fold. Metrics evaluated during CV can be accessed using the `get_metrics` function. Custom metrics can be added or removed using `add_metric` and `remove_metric` function.

In [ ]:
# ensemble with bagging
ensemble_model(dt, method = 'Bagging')

In [ ]:
# ensemble with boosting
ensemble_model(dt, method = 'Boosting')

Some other parameters that you might find very useful in `ensemble_model` are:

- choose_better
- n_estimators
- groups
- fit_kwargs
- probability_threshold
- return_train_score

You can check the docstring of the function for more info.

In [ ]:
# help(ensemble_model)

## ✅ Blend Models

In [ ]:
best_recall_models_top3 = compare_models(sort = 'Recall', n_select = 3)

In [ ]:
# blend top 3 models
blend_models(best_recall_models_top3)

Some other parameters that you might find very useful in `blend_models` are:

- choose_better
- method
- weights
- fit_kwargs
- probability_threshold
- return_train_score

You can check the docstring of the function for more info.

In [ ]:
# help(blend_models)

## ✅ Stack Models

This function trains a meta-model over select estimators passed in the estimator_list parameter. The output of this function is a scoring grid with CV scores by fold. Metrics evaluated during CV can be accessed using the `get_metrics` function. Custom metrics can be added or removed using `add_metric` and `remove_metric` function.

In [ ]:
# stack models
stack_models(best_recall_models_top3)

Some other parameters that you might find very useful in `stack_models` are:

- choose_better
- meta_model
- method
- restack
- probability_threshold
- return_train_score

You can check the docstring of the function for more info.

In [ ]:
# help(stack_models)

## ✅ Plot Model

This function analyzes the performance of a trained model on the hold-out set. It may require re-training the model in certain cases.

In [ ]:
# plot class report
plot_model(best, plot = 'class_report')

In [ ]:
# to control the scale of plot
plot_model(best, plot = 'class_report', scale = 2)

In [ ]:
# to save the plot
plot_model(best, plot = 'class_report', save=True)

Some other parameters that you might find very useful in `plot_model` are:

- fit_kwargs
- plot_kwargs
- groups
- display_format

You can check the docstring of the function for more info.

In [ ]:
# help(plot_model)

## ✅ Interpret Model

This function analyzes the predictions generated from a trained model. Most plots in this function are implemented based on the SHAP (Shapley Additive exPlanations). For more info on this, please see https://shap.readthedocs.io/en/latest/

In [ ]:
# train lightgbm model
lightgbm = create_model('lightgbm')

In [ ]:
# interpret summary model
interpret_model(lightgbm, plot = 'summary')

In [ ]:
# reason plot for test set observation 1
interpret_model(lightgbm, plot = 'reason', observation = 1)

Some other parameters that you might find very useful in `interpret_model` are:

- plot
- feature
- use_train_data
- X_new_sample
- y_new_sample
- save

You can check the docstring of the function for more info.

In [ ]:
# help(interpret_model)

## ✅ Get Leaderboard

This function returns the leaderboard of all models trained in the current setup.

In [ ]:
# get leaderboard
lb = get_leaderboard()
lb

In [ ]:
# select the best model based on F1
lb.sort_values(by='F1', ascending=False)['Model'].iloc[0]

Some other parameters that you might find very useful in `get_leaderboard` are:

- finalize_models
- fit_kwargs
- model_only
- groups

You can check the docstring of the function for more info.

In [ ]:
# help(get_leaderboard)

## ✅ AutoML
This function returns the best model out of all trained models in the current setup based on the optimize parameter. Metrics evaluated can be accessed using the `get_metrics` function.

In [ ]:
automl()

## ✅ Dashboard
The dashboard function generates the interactive dashboard for a trained model. The dashboard is implemented using `ExplainerDashboard`. For more information check out [Explainer Dashboard.](explainerdashboard.readthedocs.io)

In [ ]:
# dashboard function
dashboard(dt, display_format ='inline')

## ✅Create App
This function creates a basic gradio app for inference.

In [ ]:
# create gradio app
create_app(best)

## ✅ Create API
This function takes an input model and creates a POST API for inference.

In [ ]:
# create api
create_api(best, api_name = 'my_first_api')

In [ ]:
# !python my_first_api.py

In [ ]:
# check out the .py file created with this magic command
# %load my_first_api.py

## ✅ Create Docker
This function creates a `Dockerfile` and `requirements.txt` for productionalizing API end-point.

In [ ]:
create_docker('my_first_api')

In [ ]:
# check out the DockerFile file created with this magic command
# %load DockerFile

In [ ]:
# check out the requirements file created with this magic command
# %load requirements.txt

## ✅ Finalize Model
This function trains a given model on the entire dataset including the hold-out set.

In [ ]:
final_best = finalize_model(best)

In [ ]:
final_best

## ✅ Convert Model
This function transpiles the trained machine learning model's decision function in different programming languages such as Python, C, Java, Go, C#, etc. It is very useful if you want to deploy models into environments where you can't install your normal Python stack to support model inference.

In [ ]:
# transpiles learned function to java
print(convert_model(dt, language = 'java'))

## ✅ Deploy Model
This function deploys the entire ML pipeline on the cloud.

**AWS:**  When deploying model on AWS S3, environment variables must be configured using the command-line interface. To configure AWS environment variables, type `aws configure` in terminal. The following information is required which can be generated using the Identity and Access Management (IAM) portal of your amazon console account:

- AWS Access Key ID
- AWS Secret Key Access
- Default Region Name (can be seen under Global settings on your AWS console)
- Default output format (must be left blank)

**GCP:** To deploy a model on Google Cloud Platform ('gcp'), the project must be created using the command-line or GCP console. Once the project is created, you must create a service account and download the service account key as a JSON file to set environment variables in your local environment. Learn more about it: https://cloud.google.com/docs/authentication/production

**Azure:** To deploy a model on Microsoft Azure ('azure'), environment variables for the connection string must be set in your local environment. Go to settings of storage account on Azure portal to access the connection string required.
AZURE_STORAGE_CONNECTION_STRING (required as environment variable)
Learn more about it: https://docs.microsoft.com/en-us/azure/storage/blobs/storage-quickstart-blobs-python?toc=%2Fpython%2Fazure%2FTOC.json

In [ ]:
# deploy model on aws s3
# deploy_model(best, model_name = 'my_first_platform_on_aws',
#             platform = 'aws', authentication = {'bucket' : 'pycaret-test'})

In [ ]:
# load model from aws s3
# loaded_from_aws = load_model(model_name = 'my_first_platform_on_aws', platform = 'aws',
#                              authentication = {'bucket' : 'pycaret-test'})

# loaded_from_aws

## ✅ Save / Load Model
This function saves the transformation pipeline and a trained model object into the current working directory as a pickle file for later use.

In [ ]:
# save model
save_model(best, 'my_first_model')

In [ ]:
# load model
loaded_from_disk = load_model('my_first_model')
loaded_from_disk

## ✅ Save / Load Experiment
This function saves all the experiment variables on disk, allowing to later resume without rerunning the setup function.

In [ ]:
# save experiment
save_experiment('my_experiment')

In [ ]:
# load experiment from disk
exp_from_disk = load_experiment('my_experiment', data=data)